# 순환 신경망

순환 신경망은 순서가 있는 데이터를 입력으로 받고 같은 네트워크를 이용해 변화하는 입력에 대한 출력을 얻어낸다. 시간의 흐름에 따라 변화하고 그 변화가 의미를 갖는 데이터이다. 

## 7.1 순환 신경망의 구조

출력값이 다음 입력을 받을 때의 RNN 네트워크에도 동일하게 전달된다. 

순환 신경망은 입력과 출력의 길이에 제한이 없다. 그렇기 때문에 다양한 형태의 네트워크를 만들 수 있다. 

## 7.2 주요 레이어 정리

### SImpleRNN 레이어
활성화함수로는 tanh가 쓰인다. tanh는 -1에서 1 사이의 출력값을 반환한다. 이 자리에 ReLU같은 다른 활성화함수도 쓸 수 있다.

    rnn1 = tf.keras.layers.SimpleRNN(units=1, activation='tanh', return_sequences=True)

units는 SimpleRNN레이어에 존재하는 뉴런의 수.

return_sequences는 출력으로 시퀀스 전체를 출력할지 여부를 나타내는 옵션, 주로 여러개의 RNN 레이어를 쌓을때 쓴다.




In [ ]:
import numpy as np
# 0.0, 0.1, 0.2, 0.3을 통해 0.4를 예측하는 네트워크를 만들어보자!

X = [] 
Y = []
for i in range(6):
    # [0, 1, 2, 3], [1, 2, 3, 4]와 같은 정수의 시퀀스를 만든다
    lst = list(range(i, i+4))

    # 위에서 구한 시퀀스의 숫자들을 각각 10으로 나눈 다음 저장한다.
    # SimpleRNN에 각 타임스텝에 하나씩 숫자가 들어가기 때문에 여기서도 하나씩 분리해서 배열에 저장한다.
    X.append(list(map(lambda c: [c/10], lst)))

    #정답에 해당하는 4, 5 등의 정수 역시 앞에서처럼 10으로 나눠서 저장한다.
    Y.append((i+4)/10)

X = np.array(X)
Y = np.array(Y)
for i in range(len(X)):
    print(X[i], Y[i])

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
                             tf.keras.layers.SimpleRNN(units=10, return_sequences=False, input_shape = [4,1]),
                             tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

SimpleRNN 레이어에서 주목해야할 점은 **input_shape**이다. 여기서 [4,1]은 각각 timesteps, input_dim을 나타낸다. 

timesteps는 순환 신경망이 입력에 대해 계산을 반복하는 횟수이고

input_dim은 입력 벡터의 크기이다.

In [ ]:
model.fit(X,Y, epochs=100, verbose=0)
print(model.predict(X))

In [ ]:
print(model.predict(np.array([[[0.6], [0.7], [0.8], [0.9]]])))
print(model.predict(np.array([[[-0.1], [0.0], [0.1], [0.2]]])))

### LSTM 레이어

SimpleRNN 레이어의 치명적인 단점은 입력 데이터가 길어질수록 학습 능력이 떨어진다. 

입력 데이터와 출력 사이의 길이가 멀어질수록 연관 관계가 작아진다는 것이다. 



In [ ]:
# 예시) 곱셈문제

X = []
Y = []
for i in range(3000):
# 0 ~ 1 범위의 랜덤한 숫자 100개를 만든다.
    lst = np.random.rand(100)

    #마킹할 숫자 2개의 인덱스를 뽑는다.
    idx = np.random.choice(100, 2, replace=False)
    
    #마킹 인덱스가 저장된 원-핫 인코딩 벡터를 만든다
    zeros = np.zeros(100)
    zeros[idx] = 1
    
    # 마킹 인덱스와 랜덤한 숫자를 합쳐서 X에 저장한다.
    X.append(np.array(list(zip(zeros, lst))))

    # 마킹 인덱스가 1인 값만 서로 곱해서 Y에 저장한다.]
    Y.append(np.prod(lst[idx]))

print(X[0], Y[0])

In [ ]:
model = tf.keras.Sequential([
                             tf.keras.layers.SimpleRNN(units=30, return_sequences=True, input_shape=[100,2]),
                             tf.keras.layers.SimpleRNN(units=30),
                             tf.keras.layers.Dense(1)
])

위 모델에서 return_sequences = True로 되어있다.  return_sequences는 레이어의 출력ㅇ르 다음 레이어로 그대로 넘겨주게 된다.

In [ ]:
model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
X = np.array(X)
Y = np.array(Y)

# 2560 개의 데이터만 학습시키나. 검증 데이터는 20%로 지정
history = model.fit(X[:2560], Y[:2560], epochs=100, validation_split = 0.2)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.legend()
plt.show()

In [ ]:
model.evaluate(X[2560:], Y[2560:])
prediction = model.predict(X[2560:2560+5])

for i in range(5):
    print(Y[2560 + i], '\t', prediction[i][0], '\tdiff:', abs(prediction[i][0] - Y[2560+i]))

prediction = model.predict(X[2560:])
fail = 0
for i in range(len(prediction)):
    if abs(prediction[i][0] - Y[2560+i]) > 0.04:
        fail += 1

print('correctness:', (440 - fail) / 440 * 100, '%')

SimpleRNN으로 만들어 보았을 때 전체적인 loss는 0.0484가 나왔다. 위에서 본 100번째 에포크의 val_loss인 0.0474보다 높으므로 네트워크가 학습과정에서 한번도 못 본 테스트 데이터에 대해서는 예측을 잘 하지 못한다. 5개의 테스트 데이터에 대한 정확도가 10.9%으로 나온다.

그렇다면 LSTM은 어떨까?


In [ ]:
model = tf.keras.Sequential([
                             tf.keras.layers.LSTM(units=30, return_sequences=True, input_shape=[100,2]),
                             tf.keras.layers.LSTM(units=30),
                             tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
X = np.array(X)

In [ ]:
Y = np.array(Y)
history = model.fit(X[:2560], Y[:2560], epochs=100, validation_split=0.2)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.legend()
plt.show()

In [ ]:
model.evaluate(X[2560:], Y[2560:])
prediction = model.predict(X[2560:2560+5])

for i in range(5):
    print(Y[2560 + i], '\t', prediction[i][0], '\tdiff:', abs(prediction[i][0] - Y[2560+i]))

prediction = model.predict(X[2560:])
fail = 0
for i in range(len(prediction)):
    if abs(prediction[i][0] - Y[2560+i]) > 0.04:
        fail += 1

print('correctness:', (440 - fail) / 440 * 100, '%')

LSTM이 훨씬 높은 정확도 92,5%를 보이고 loss도 0.04보다 높은게 1도 없음

### GRU 레이어

LSTM 레이어와 비슷한 역할을 하지만 계산 구조가 훨씬 간단.

앞서 했던 문제를 이번엔 GRU 레이어로 풀어보겠음

In [ ]:
model = tf.keras.Sequential([
                             tf.keras.layers.GRU(units=30, return_sequences=True, input_shape = [100,2]),
                             tf.keras.layers.GRU(units=30),
                             tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

위 곱셈 문제를 풀기 위한 네트워크 파라미터 수

SimpleRNN 2851개
LSTM 11311개
GRU 8671개개

In [ ]:
X = np.array(X)
Y = np.array(Y)
history = model.fit(X[:2560], Y[:2560], epochs=100, validation_split=0.2)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.legend()
plt.show()

In [ ]:
model.evaluate(X[2560:], Y[2560:])
prediction = model.predict(X[2560:2560+5])

for i in range(5):
    print(Y[2560 + i], '\t', prediction[i][0], '\tdiff:', abs(prediction[i][0] - Y[2560+i]))

prediction = model.predict(X[2560:])
cnt = 0
for i in range(len(prediction)):
    if abs(prediction[i][0] - Y[2560+i]) > 0.04:
        cnt += 1

print('correctness:', (440 - cnt) / 440 * 100, '%')

### 임베딩 레이어

임베딩 레이어는 자연어를 수치화 정보로 바꾸기 위한 레이어이다.

임베딩 레이어보다 좀 더 쉬운 기법은 자연어를 구성하는 단위에 대해 정수 인덱스를 저장하는 방법이다. 단어를 기반으로 정수 인덱스를 저장하여 사용하는 것이다. 그 다음 이를 원-핫 인코딩을 이요하여 0과 1로만 이루어진 열로 바꾸어준다.

인덱스를 사용하는 원-핫 인코딩 방식의 단점은 메모리의 양에 비해 너무 적은 정보량을 사용한다는 것이다. 또한 인덱스에 저장된 단어의 수가 많아질수록 원-핫 인코딩 배열의 두번째 차원의 크기도 비례해서 늘어난다는 것이다.

원-핫 인코딩 방식에 비해 임베딩 레이어는 한정된 길이의 벡터로 자연어의 구성 단위인 자소, 문자, 단어, ngram 등을 표현할 수 있다.

## 7.3 긍정, 부정 감성 분석

In [ ]:
import tensorflow as tf
import numpy as np
    

path_to_train_file = tf.keras.utils.get_file('train.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
path_to_test_file = tf.keras.utils.get_file('test.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

In [ ]:
# 데이터를 메모리에 불러온다. 인코딩 형식으로 utf-8을 지정해야한다.
train_text = open(path_to_train_file, 'rb').read().decode(encoding='utf-8')
test_text = open(path_to_test_file, 'rb').read().decode(encoding='utf-8')

# 텍스트가 총 몇자인지 확인한다.

print('train text 길이 : {}'.format(len(train_text)))
print('test text 길이 : {}'.format(len(test_text)))
print() 

print(train_text[:300])

데이터의 각 행은 '\t'으로 구분되어 있다. 각 id는 각 데이터의 고유한 id 이고 document는 실제 리뷰 내용이다. label은 긍.부정 내용이다.

In [ ]:
train_Y = np.array([[int(row.split('\t')[2])] for row in train_text.split('\n')[1:] if row.count('\t') > 0])
test_Y = np.array([[int(row.split('\t')[2])] for row in test_text.split('\n')[1:] if row.count('\t') > 0])

print(train_Y.shape, test_Y.shape)
print(train_Y[:5])

그 다음 입력으로 쓸 자연어를 토큰화하고 정제해야한다. 토큰화로는 단어를 사용할 것이기 때문에 띄어쓰기를 단위로 나누면 된다. 정제란 원하지 않는 입력이나 불필요한 기호 등ㅇ르 제거하는 것이다. 정제를 위해 함수로는 김윤 박사의 CNN_sentence 깃허브 저장소의 코드를 사용한다. 

In [ ]:
import re
# From https://github.com/yoonkim/CNN_sentence/blob/master/process_daata.py

def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Every dataset is lower cased except for TREC
    """
    string = re.sub(r"[^가-힣A-Za-z0-9(),!?\'\`]", " ", string)     
    string = re.sub(r"\'s", " \'s", string) 
    string = re.sub(r"\'ve", " \'ve", string) 
    string = re.sub(r"n\'t", " n\'t", string) 
    string = re.sub(r"\'re", " \'re", string) 
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r",", " , ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " \( ", string) 
    string = re.sub(r"\)", " \) ", string) 
    string = re.sub(r"\?", " \? ", string) 
    string = re.sub(r"\s{2,}", " ", string)    
    string = re.sub(r"\'{2,}", "\'", string)
    string = re.sub(r"\'", "", string)

    return string.lower()

train_text_X = [row.split('\t')[1] for row in train_text.split('\n')[1:] if row.count('\t') > 0]
train_text_X =[clean_str(sentence) for sentence in train_text_X]
# 문장을 띄어쓰기 단위로 단어 분리
sentences = [sentence.split(' ') for sentence in train_text_X]
for i in range(5):
    print(sentences[i])

    string = re.sub(r"[^가-힣A-Za-z0-9(),!?\'\`]", " ", string)     

다음 첫 줄의 의미는 한글, 영어, 숫자, 괄호, 쉼표, 느낌표, 물음표, 작은 따움표, 역 따옴표를 제외한 모든 것ㅇ르 공백으로 바꾸겠다는 의미.

훈련 데이터의 처음 다섯개를 출력해보면 구두점(.)같은 기호가 삭제된 것을 알 수 있다.

하지만 네트워크에 입력하려면 데이터의 크기가 동일해야한다. 하지만 현재의 각 문장의 길이가 다르기 때문에 문장의 길이를 맞춰야 한다. 이를 위해 적당한 길이의 문장이 어느정도인지 확인하고 긴 문장은 줄이고 짧은 문장에는 공백을 넣는 패딩을 채운다. 

In [ ]:
import matplotlib.pyplot as plt
sentence_len = [len(sentence) for sentence in sentences]
sentence_len.sort()
plt.plot(sentence_len)
plt.show()

print(sum([int(l<=25) for l in sentence_len]))

y축이 문장의 단어 개수일 때 15만개의 문장 중에서 대부분 40단어 이하로 구성되어있다.  특히 25단어 이하인 문장의 수는 142,587개로 전체의 95%정도이다. 

따라서 기분이 되는 문장의 길이를 25개로 잡고 이 이상은 생략, 이 이하는 패딩으로 25로 맞춰주면 임베딩 레이어에 넣을 준비가 끝난다.

또 하나 처리해야하는 부분은 각 단어의 최대 길이를 조정하는 일이다.


In [ ]:
sentences_new = []
for sentence in sentences:
    sentences_new.append([word[:5] for word in sentence][:25])
sentences = sentences_new
for i in range(5):
    print(sentences[i])

이제는 한 문장의 길이를 25으로 바꾸기 위해서 tf.keras에서 제공하는 pad_sequences를 사용한다.

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(sentences)
train_X = tokenizer.texts_to_sequences(sentences)
train_X = pad_sequences(train_X, padding='post')

print(train_X[:5])

Tokenizer는 데이터에 출현하는 모든 단어의 개수를 세고 빈도수를 정렬해서  num_words에 지정된 만큼만 숫자로 반환하고 나머지는 0으로 반환한다.

tokenizer.fit_on_texts(sentences)는 tokenizer에 데이터를 실제로 입력한다.

이 과정을 거친 뒤 tokenizer.texts_to_sequences(sentences)는 문장을 입력받아 숫자를 반환한다. 마지막으로 pad_sequences()는 입력된 데이터에 패딩을 더한다.

pad_sequences()의 padding 인수는 2가지 옵션이 있다. 이 중 'pre'는 문장 앞에 패딩을 넣고 'post'는 문장 뒤에 패딩을 넣는다.

단어가 빈도수에서 상위 20,000개 안에 들지 못하면 0으로 처리된다. 

In [ ]:
#Tokenizer 예시
print(tokenizer.index_word[19999])
print(tokenizer.index_word[20000])
temp = tokenizer.texts_to_sequences(['#$#$#', '경우는', '잊혀질', '연기가'])
print(temp)
temp = pad_sequences(temp, padding='post')
print(temp)

위에서  Tokenizer.index_word에 저장돼 있는 19,999번째 단어와 20,000번째 단어를 확인해본 뒤, 이 단어들로 구성된 문장을  Tokenizer에 넣어보았다.

texts_to_sequences()에서 반환하는 데이터는 19,999번째 단어인 "겨우는"과 "연기가"만 의미 있는 데이터로 남고 나머지는 공백으로 반환한다.

이 공백은 pad_sequences()를 통과하면서 0으로 바뀐다.

여기서 pad_sequences()의 maxlen인수가 지정되지 않았기 때문에 입력된 문장 전체 길이 중 가장 긴 길이로 문장의 길이를 맞춘다. 입력 문장이 하나이기 때문에 입력과 출력 문장의 길이는 동일하다.

실제로 네트워크를 정의하고 학습시켜보자.

먼저 임베딩 레이어와 LSTM 레이어를 연결한 뒤 마지막에 Dense 레이어의 소프트맥스 활성화함수를 사용해 긍정/부정을 분류하는 네트워크를 정의해보자

In [ ]:
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(20000, 300, input_length=25),
                             tf.keras.layers.LSTM(units=50),
                             tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

위의 네트워크에서 임베딩 레이어는 시퀀셜 모델의 첫 레이어이기 때문에 입력 형태에 대한 정의가 필요하다. input_length 인수를 25로 지정해 각 문장에 들어있는 25개 단어를 길이 300의 임베딩 벡터로 변환한다.

네트워크의 loss는 sparse_categorical_crosstropy를 사용했다. 여러 개의 정답 중 하나를 맞추는 분류에서는 categorical_crossentropy를 사용하고, sparse는 정답인 Y가 희소행렬일때 사용한다!

In [ ]:
history = model.fit(train_X, train_Y, epochs = 5, batch_size=128, validation_split=0.2)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], 'g-', label='accuracy')
plt.plot(history.history['val_accuracy'], 'k--', label='val_accuracy')
plt.legend()

plt.show()

val_loss가 꾸준히 증가하고 val_accuracy도 점점 떨어지고 있다. == 과적합 되었따 

과적합의 이유는 ㅇ미베딩 레이어를 랜덤한 값에서부터 시작해서 학습시키기 때문에 각 단어를 나타내는 벡터의 품질이 좋지 않아서이다.

이를 개선하기 위한 방법으로 임베딩 레이어를 별도로 학습시켜서 네트워크에 불러와서 사용하거나, CNN을 사용하는 방법이 있다.

학습된 네트워크가 테스트 데이터를 어떻게 평가하는지 확인하기 위해 test_text에도  train_text와 같은 변환 과정을 거쳐서 test_X를 만들고 model.evaluate()로 평가해 본다.

이때 주목해야할 점은 train_X를 만들 때 학습시켰던 Tokenizer를 어떤 변경 없이 그대로 사용하고 있다는 것이다. 훈련 데이터와는 다르게 테스트 데이터에서는 어떤 단어가 나타날지 모르기 때문에 Tokenizer는 훈련 데이터로만 학습시켜야 한다. 

In [ ]:
test_text_X = [row.split('\t')[1] for row in test_text.split('\n')[1:] if row.count('\t') > 0]
test_text_X = [clean_str(sentence) for sentence in test_text_X]
sentences= [sentence.split(' ') for sentence in test_text_X]
sentences_new = []

for sentence in sentences:
    sentences_new.append([word[:5] for word in sentence][:25])
sentences = sentences_new

test_X = tokenizer.texts_to_sequences(sentences)
test_X = pad_sequences(test_X, padding='post')

model.evaluate(test_X, test_Y, verbose=0)

In [ ]:
test_sentence = '재밌을 줄 알았는데 완전 실망했다. 너무 졸리고 돈이 아까웠다.'
test_sentence = test_sentence.split(' ')
test_sentences = []
now_sentence = []
for word in test_sentence:
    now_sentence.append(word)
    test_sentences.append(now_sentence[:])

test_X_1 = tokenizer.texts_to_sequences(test_sentences)
test_X_1 = pad_sequences(test_X_1, padding='post', maxlen = 25)
prediction = model.predict(test_X_1)
for idx, sentence in enumerate(test_sentences):
    print(sentence)
    print(prediction[idx])

출력은 문장의 변화에 따른 감정 분석 예측 결과이다. 처음에는 "재미있을"이라는 단어만 입력됐을 때는 긍정의 확률이 높으나, 다른 단어들이 입력되면서 줄어들었다. 

## 자연어 생성

### 단어 단위 생성

In [ ]:
path_to_file = tf.keras.utils.get_file('input.txt', 'http://bit.ly/2Mc3SOV')

In [ ]:
# 데이터를 메모리에 불러온다.
train_text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# 텍스트가 총 몇 자인지 확인한다.
print('Length of text : {}'.format(len(train_text)))
print()

#처음 1100자를 확인한다.
print(train_text[:100])

In [ ]:
import re
# From https://github.com/yoonkim/CNN_sentence/blob/master/process_daata.py

def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Every dataset is lower cased except for TREC
    """
    string = re.sub(r"[^가-힣A-Za-z0-9(),!?\'\`]", " ", string)     
    string = re.sub(r",", " , ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", "", string) 
    string = re.sub(r"\)", "", string) 
    string = re.sub(r"\?", " \? ", string) 
    string = re.sub(r"\s{2,}", " ", string)    
    string = re.sub(r"\'{2,}", "\'", string)
    string = re.sub(r"\'", "", string)

    return string

train_text = train_text.split('\n')
train_text = [clean_str(sentence) for sentence in train_text]
train_text_X = []
for sentence in train_text:
    train_text_X.extend(sentence.split(' '))
    train_text_X.append('\n')

train_text_X = [word for word in train_text_X if word != '']

print(train_text_X[:30])

다음으로 할 작업은 단어를 토큰화 하는 것이다. 여기서 직접 토큰화를 해보도록 하자.

In [ ]:
# 단어의 set을 만든다.
vocab = sorted(set(train_text_X))
vocab.append('UNK')
print(' {} unique words'.format(len(vocab)))

# vocab list를 숫자로 매핑하고, 반대도 실행한다.
word2idx = {u:i for i,u in enumerate(vocab)}
idx2word = np.array(vocab)

text_as_int = np.array([word2idx[c] for c in train_text_X])

# word2idx 의 일부를 알아보기 쉽게 출력해보자.
print('{')
for word,_ in zip(word2idx, range(10)):
    print(' {:4s}: {:3d},'.format(repr(word), word2idx[word]))
print('...\n}')

print('index of UNK: {}'.format(word2idx['UNK']))

2번째 줄에서 텍스트에 들어간 단어가 중복되지 않는 리스트를 만든 다음, 3번째 줄에서 텍스트에 존재하지 않는 토큰을 나타내는 'UNK'를 넣는다. 총 단어수는 332,640이고, 이중 'UNK'의 인덱스는 332,639이다. 이 인덱스는 나중에 임의의 문장을 입력할 때 텍스트에 미리 준비돼 있지 않았던 단어를 쓸 수 있기 때문에 그에 대한 토큰으로 쓰일 것이다. 

In [ ]:
print(train_text_X[:20])
print(text_as_int[:20])

이제 학습을 위한 데이터 셋을 만든다. 여기서 기존의 train_X train_Y 를 넘파이 array로 만드는 방식이 아닌, tf.data.Dataset을 이용한다. Dataset의 장점은 간단한 코드로 데이터 섞기, 배치수 만큼 자르기, 다른 Dataset에 매핑하기 등을 수행할 수 있다는 점이다.

In [ ]:
seq_length = 25
examples_per_epoch = len(text_as_int) // seq_length
sentence_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sentence_dataset = sentence_dataset.batch(seq_length+1, drop_remainder = True)
for item in sentence_dataset.take(1):
    print(idx2word[item.numpy()])
    print(item.numpy())

seq_length를 25로 ㅅ걸정하여 25개의 단어가 주어졌을 때 다음단어를 예측하도록 데이터를 만들려고 한다.

    sentence_dataset = tf.dat.Dataset.from_tensor_slices(text_as_int)
 
Dataset를 생성하는 코드는 위와 같이 한 줄로 간단하다.

    sentence_dataset = sentence_dataset.batch(seq_length+1, drop_remainder=True)

Dataset에 쓰이는 batch() 함수는 Dataset에서 한번에 반환하는 데이터의 숫자를 지정한다. 여기서는 seq_length+1을 지정했는데, 처음의 25개의 단어와 그 뒤에 오는 정답이 도리 1단어를 합쳐서 함께 반환하기 위해서이다. 

또 drop_remainder=True 옵션으로 남는 부분은 버린다. 출력에서 의도한 대로 26 단어가 반환되는 것을 확인할 수 있다.

이렇게 만들어진 Dataset으로 새로운 Dataset을 만들어 본다. 

26개의 단어가 각각 입력과 정받으로 묶여서 ([25단어],1단어) 형태의 데이터를 반환하게 만드는 Dataset을 만든다.


In [ ]:
def split_input_target(chunk):
    return[chunk[:-1], chunk[-1]]

train_dataset = sentence_dataset.map(split_input_target)
for x,y in train_dataset.take(1):
    print(idx2word[x.numpy()])
    print(x.numpy())
    print(idx2word[y.numpy()])
    print(y.numpy())

먼저 26개의 단어를 25단어, 1단어로 잘라주는 함수를 정의한 다음, 이 함수를 기존 sentence_dataset에 map()함수를 이요해 새로운 Dataset인 train_dataset을 만든다.

그 다음으로 Dataset의 데이터를 섞고 batch_size를 다시 설정한다.

In [ ]:
BATCH_SIZE = 128
setps_per_epoch =  examples_per_epoch // BATCH_SIZE
BUFFER_SIZE = 10000

train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

빠른 학습을 위해 한 번에 128개의 데이터를 학습하게 하고, 데이터를 섞을 때의 BUFFER_SIZE는 10000으로 설정한다.

tf.data는 이론적으로 무한한 데이터에 대해 대응 가능하기 때문에 한번에 모든 데이터를 섞지 않는다. 따라서 버퍼에 일정한 양의 데이터를 올려놓고 섞는데, 그 사이즈를10000으로 설정한 것이다. 

In [ ]:
total_words = len(vocab)
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(total_words, 100, input_length=seq_length),
                             tf.keras.layers.LSTM(units=100, return_sequences=True),
                             tf.keras.layers.Dropout(0.2),
                             tf.keras.layers.LSTM(units=100),
                             tf.keras.layers.Dense(total_words, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def testmodel(epoch, logs):
    if epoch % 5 != 0 and epoch != 49:
        return
    test_sentence = train_text[0]

    next_words = 100
    for _ in range(next_words):
        test_text_X = test_sentence.split(' ')[-seq_length:]
        test_text_X = np.array([word2idx[c] if c in word2idx else word2idx['UNK'] for c in test_text_X])
        test_text_X = pad_sequences([test_text_X], maxlen=seq_length, padding='pre', value=word2idx['UNK'])

        output_idx = model.predict_classes(test_text_X)
        test_sentence += ' ' + idx2word[output_idx[0]]

    print()
    print(test_sentence)
    print()

testmodelcb = tf.keras.callbacks.LambdaCallback(on_epoch_end=testmodel)

history = model.fit(train_dataset.repeat(), epochs=50, steps_per_epoch = setps_per_epoch, callbacks=[testmodelcb], verbose=2)